# **Google Colab TPU Kulanarak Shakespeare Kestirimi  (LSTM )**



---


**Kaynak:** *[Google Colab -Tutorial](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb)*

[<img align="right" width="100" height="100" src="http://www.i2symbol.com/images/symbols/style-letters/circled_latin_capital_letter_a_u24B6_icon_128x128.png">](https://www.ayyucekizrak.com/)

## Veri İndirme

*The Complete Works of William Shakespeare* yazı formatında veriyi [Project Gutenberg](https://www.gutenberg.org/) adresinden indirme işlemi. Dosya içinden paragrafları *eğitim seti* için kullanacağız. *Hedef* için paragraflar birer karakterle ifade edilmektedir.

In [0]:
!wget --show-progress --continue -O /content/shakespeare.txt http://www.gutenberg.org/files/100/100-0.txt

--2018-10-24 11:03:37--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852404 (5.6M) [text/plain]
Saving to: ‘/content/shakespeare.txt’

/content/shakespear 100%[===================>]   5.58M  5.54MB/s    in 1.0s    

2018-10-24 11:03:39 (5.54 MB/s) - ‘/content/shakespeare.txt’ saved [5852404/5852404]



### Veri Üretici Oluşturma

In [0]:
import numpy as np
import six
import tensorflow as tf
import time
import os

# Bu adres, TensorFlow'u yapılandırırken kullanacağımız TPU'yu tanımlar.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

SHAKESPEARE_TXT = '/content/shakespeare.txt'

tf.logging.set_verbosity(tf.logging.INFO)

def transform(txt, pad_to=None):
  # ascii olmayan karakterleri bırakır
  output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  if pad_to is not None:
    output = output[:pad_to]
    output = np.concatenate([
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output

def training_generator(seq_len=100, batch_size=1024):
  """A generator yields (source, target) arrays for training."""
  with tf.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    txt = f.read()

  tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
  source = transform(txt)
  while True:
    offsets = np.random.randint(0, len(source) - seq_len, batch_size)

    # Modelimiz seyrek çapraz entropi kaybı kullanmaktadır, 
    # ancak Keras, etiketlerin giriş kayıtlarıyla aynı sıraya sahip olmasını gerektirir. 
    # Bunu açıklamak için boş bir son boyut ekliyoruz.
    
    yield (
        np.stack([source[idx:idx + seq_len] for idx in offsets]),
        np.expand_dims(
            np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
            -1),
    )

six.next(training_generator(seq_len=10, batch_size=1))

INFO:tensorflow:Input text [5834393] ﻿
Project Gutenberg’s The Complete Works of Willi


(array([[110, 100,  32, 100, 105, 101,  33,  13,  10,  32]], dtype=int32),
 array([[[100],
         [ 32],
         [100],
         [105],
         [101],
         [ 33],
         [ 13],
         [ 10],
         [ 32],
         [ 32]]], dtype=int32))

## Model Oluşturma

Model, iki katmanlı, ileri-LSTM olarak tanımlanmıştır - tf.keras standart LSTM tanımından iki değişiklikle:

1. Modelimizin [XLA compiler](https://www.tensorflow.org/performance/xla/) derleyicisinin statik şekil gereksinimini karşılayan giriş `shape`ini tanımlayın..
2. Keras optimizer yerine `tf.train.Optimizer` kullanın.(Keras optimizer desteği hala deneysel).

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
  return model

## Model Eğitimi

`tf.keras` fonksiyonu bu aşamada TPU kullanımının sağlanması için `tf.contrib.tpu.keras_to_tpu_model` şeklinde güncellenmesi gerekiyor.
Ardından Keras'ta standart eğitim işleminde olduğu gibi `fit`, `predict`, ve `evaluate` kullanılıyor.

In [0]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=10,
)
tpu_model.save_weights('/tmp/bard.h5', overwrite=True)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.16.57.82:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8967059024566676383)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4606660762033467324)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 11217221221209779883)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3917334524445245232)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4692620694230546906)
INFO:tensorflow:*** Available Device: _DeviceAt

## Model Kestirimlerini Hesaplayın

Tahminleri yapmak ve kendi Shakespeare-esque eserinizi oluşturmak için eğitimli modeli kullanın.
Modeli bir başlangıç (seed sentence) cümlesi ile başlatın, daha sonra 250 karakterden oluşturun.
Başlangıç cümlesinden beş tahmin yapacağız.

In [0]:
BATCH_SIZE = 5
PREDICT_LEN = 250

# Keras, durum bilgisi olan modeller için toplu boyutun önceden belirlenmesini gerektirir.
# Her seferinde bir karakterde besleyeceğimiz ve bir sonraki karakteri tahmin edeceğimiz için 
# 1'lik bir dizi uzunluğu kullanırız.

prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# Modeli ilk string'i kullanarak ekiyoruz. BATCH_SIZE sayısı kadar kopyalıyoruz.

seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# Öncelikle, modelin durumunu başlatmak için seed çalıştırın. 
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Şİmdi kestirimleri yapmaya başlayabiliriz!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # Çıkış dağılımından örnekler
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 They gave me
    one keeper.
  SHALLOW. Belike a sort, you shall thrift, if I be my liege;
    She is one soul forswear, or she comes
    To her both together it is, the heavens
    Or to the seat; cheer at me,
    Wherein I shall make my hear

PREDICTION 1


 In that rude honour,
    With grievous Greeks here, for eat my arm;
    For you call herself fire him, loathd our knees
    In my hopes, fowls read melies here a controving,
    Have the provision. Make them here. O, let me stay to deliver
    

PREDICTION 2


 And Flesh no man
    Worse this light, we'll drown thy love;
  But that hath tongue thoughts churl nothing, I
To Talbot; this is what to rack this gap in Captain;
Made our torched bachwith, coals (burn
And make me swear. What, shall we go in?


PREDICTION 3


 he fought
    me; and therefore let the treasure marks
    Like a cold and all but my hands.
    Ere I ceer the gold affectiond holy fair glory
    That hopes may gain defacd, wh'right was nothing,
  